In [26]:
%load_ext autoreload
% autoreload 2
import setup_deep_q_expm
import numpy as np
import tensorflow as tf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
from expm_params import expm_params

In [28]:
expm_name = 'CartPole-v1'

log,writer,merged,sampler,algo,qnet,policy,run_params = setup_deep_q_expm.setup_expm(expm_name = expm_name,params = expm_params)

//anaconda/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [29]:
expm_params[expm_name]

{'algorithm': {'discount': 0.95, 'lr': 0.001, 'tau': 0.001},
 'base': {'env_name': 'CartPole-v1',
  'epoch_length': 400,
  'grad_steps_per_t': 1,
  'max_episode_length': -1,
  'max_epochs': 5000,
  'online_training': False},
 'nnet': {'n_hidden': 24, 'n_layers': 2},
 'policy': {'epsilon_decay': 0.99,
  'epsilon_end': 0.01,
  'epsilon_start': 1.0,
  'reward_scale': 1,
  'scheme': 'Epsilon'},
 'replay_buffer': {'batch_size': 32,
  'max_buffer_size': 2000,
  'min_pool_size': 32}}

In [30]:
for i in range(run_params['max_epochs']):
    sampler.reset()
    epoch_avg_losses = 0

    for t in range(run_params['epoch_length']):
        sampler.sample()
        if sampler.batch_ready() or run_params['online_training']:
            if run_params['online_training']:
                samples = sampler.get_last_sample()
                summary,losses,qnet_o = algo.train(samples,merged,algo.Q_Loss,qnet.output) 
            else:
                for j in range(run_params['grad_steps_per_t']):
                    samples = sampler.get_samples()
                    
                    qnet_next_obs = qnet.output.eval(feed_dict = {qnet.obs : samples['next_observations']})
                    pred = np.argmax(qnet_next_obs,axis=1)
                    one_hot = np.eye(qnet.output_size)[pred]
                    tqnet_output = algo.target_Q_outputs.eval(feed_dict = {algo.next_obs : samples['next_observations']})
#                      np.sum(one_hot*tqnet_output,axis=1)
                    doubleQ = tqnet_output[range(expm_params[expm_name]['replay_buffer']['batch_size']),pred]
                    
                    qt = samples['rewards'] + algo.discount * (1- samples['dones']) *doubleQ
                    feed_dict = algo._construct_feed_dict(samples)
                    feed_dict[algo.Q_t] = qt
                          
                    _,summary,losses,qnet_o = tf.get_default_session().run([algo.train_ops,merged,algo.Q_Loss,qnet.output], feed_dict = feed_dict)

#                     summary,losses,qnet_o= algo.train(samples,merged,algo.Q_Loss,qnet.output) 
            epoch_avg_losses = (epoch_avg_losses*(t) + np.array(losses))/(t+1)
    print('-----')
    print(np.sqrt(np.mean((qnet_next_obs[:10]-tqnet_output[:10])**2)))
                    
    print(qnet_o[0])
    log.record('mean_episode_reward',sampler.mean_episode_reward)
    writer.add_summary(summary, i)
    print(epoch_avg_losses)

    writer.flush()

    print('Epoch %i, mean_reward %d' % (i, sampler.mean_episode_reward))



-----
238.99481
[-148.90883 -104.28388]
48.83081581771373
Epoch 0, mean_reward 10
-----
2281.8364
[-3133.4287 -2170.8362]
1206.6281198883046
Epoch 1, mean_reward 9
-----
9020.956
[-7895.4834 -5331.7456]
5513.362417602539
Epoch 2, mean_reward 9
-----
19511.174
[-17726.541 -11747.774]
14236.248415527343
Epoch 3, mean_reward 9
-----
36914.543
[-54999.562 -35546.   ]
27535.736103515625
Epoch 4, mean_reward 9
-----
59962.383
[-67423.92  -39435.918]
46811.96973632815
Epoch 5, mean_reward 9
-----
69085.8
[-117652.33  -62713.45]
70933.10579101561
Epoch 6, mean_reward 9
-----
92643.836
[-168463.69   -84549.875]
101252.9239453125
Epoch 7, mean_reward 9
-----
112054.82
[-182398.31  -86550.45]
136835.39191406255
Epoch 8, mean_reward 9
-----
108720.56
[-162322.28  -73668.98]
177529.65681640626
Epoch 9, mean_reward 9


KeyboardInterrupt: 